In [1]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import lightgbm as lgbm
import warnings
warnings.filterwarnings('ignore')

C:\Users\samsung\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 깨달음
 - 무식하게 하지말자
 - 많다고 좋은거 아니다
 - 알고리즘은 맞는거 써야한다

## 반성
 - 변수 조절 거의 안했다. 잘 좀 하자
 - 전처리에서 틀리면 다 틀린거다

## --Test2--</br>
 1. Test three parameters </br>
 2. After check submission, record the LB score </br>
 3. Stacking regard of the ratio </br>


In [16]:
train = pd.read_csv('train_set2.csv')
test = pd.read_csv('test_set2.csv')
idSeries = np.array(test['id'].values)

In [17]:
print(len(idSeries))
print(train.shape, test.shape)

625134
(1450352, 26) (625134, 24)


In [18]:
set(train.columns) - set(test.columns)

{'dropoff_datetime', 'trip_duration'}

In [19]:
train.drop(['id','pickup_datetime', 'dropoff_datetime'], axis = 1, inplace = True)
test.drop(['id','pickup_datetime'], axis = 1, inplace = True)
train, test = dummyBuilding(train.copy(), test.copy())
print(train.shape, test.shape)

(1450352, 261) (625134, 262)


In [6]:
"""
ref = dict()
for itr, x in enumerate(train['condition'].unique()):
    ref[x] = itr
train.drop(['id','pickup_datetime', 'dropoff_datetime'], axis = 1, inplace = True)
test.drop(['id','pickup_datetime'], axis = 1, inplace = True)
ix = 0
for df in [train, test]:
    df['condition']  = df['condition'].map(ref)
if train.condition.nunique() == test.condition.nunique(): print('Go!')
"""

Go!


In [20]:
colsample_bytree = [0.6, 0.7, 0.8]
for itr, bytree in enumerate(colsample_bytree):
    subtitle = 'submission_ver_' + str(itr) + '.csv'
    paramSet = {
    'boosting_type': 'gbdt', 'objective': 'regression', 'nthread': -1,
    'num_leaves': 32, 'learning_rate': 0.03, 'max_depth': -1,
    'max_bin': 255, 'subsample_for_bin': 50000, 'metric': 'l2_root',
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': bytree, 'reg_alpha': 1, 'reg_lambda': 1.2,
    'min_split_gain': 0.6, 'min_child_weight': 4, 'min_child_samples': 10, 'scale_pos_weight': 1}
    
    prediction = lgbmTest(train, test, paramSet)
    #prediction = np.exp(prediction) - 1
    tmpSub = pd.DataFrame({'id':idSeries, 'trip_duration':prediction})
    if tmpSub.shape[0] == len(idSeries):
        print('itr_th submission')
        tmpSub.to_csv(subtitle, index = False)
        
    else:
        print('Different Length')
        break
    

itr_th submission
itr_th submission
itr_th submission


### Stacking
동일 비율로 Stacking 시도

In [38]:
res = []
for itr in range(3):
    subtitle = 'submission_ver_' + str(itr) + '.csv'
    sub = pd.read_csv(subtitle)
    res.append(np.array(sub['trip_duration'].values))
    if itr == 0: idSeries = sub['id'].values
    del sub
res = np.array(res)
meanPrediction = res.mean(axis=1)
tmpSub = pd.DataFrame({'id':idSeries, 'trip_duration':prediction})
tmpSub.to_csv('submission_StackMean.csv', index=  False)

최종결과 0.8984, 거의 꼴등

In [3]:
def lgbmTest(train, test, lgbm_params):
    #train['trip_duration'] = np.log1p(train['trip_duration'])
    Y_train = train['trip_duration'].copy()
    train = train.drop(['trip_duration'], axis =1)
    cv_sum = 0
    fpred = []
    lgbm_rounds = []
    NFOLDS = 5
    cv_score_list = []
    
    """
    kf = KFold(train.shape[0], n_folds = NFOLDS)
    for ix, (train_index, cross_index) in enumerate(kf):
        print("{} Fold \n".format(ix))
        x_train, x_cross = train.iloc[train_index], train.iloc[cross_index]
        y_train, y_cross = Y_train.iloc[train_index], Y_train.iloc[cross_index]
    
        dtrain = lgbm.Dataset(x_train, label = y_train.values, silent = True)
        dvalid = lgbm.Dataset(x_cross, label = y_cross.values, silent = True)
        model = lgbm.train(lgbm_params, train_set = dtrain, num_boost_round = 100000, valid_sets = dvalid,
                      early_stopping_rounds = 100, verbose_eval = None,)
        lgbm_rounds.append(model.best_iteration)
        scores_val = model.predict(x_cross, num_iteration = model.best_iteration)
        cv_score = sqrt(mean_squared_error(y_cross, scores_val))
        print('eval-MAE : %.6f' % cv_score)
        cv_score_list.append(cv_score)
        cv_sum = cv_sum + cv_score
    
    score = cv_sum / NFOLDS
    print("Average LogLikelihood: ", score)
    for cv_index, cv_s in enumerate(cv_score_list):
        print(cv_index+1, cv_s)
    
    print('test prediction')
    bstRound = max(lgbm_rounds)
    print('bst Round :', bstRound)
    """
    dtrain_all = lgbm.Dataset(train, label = Y_train.values, silent = True)
    model = lgbm.train(lgbm_params, dtrain_all)
    predictions = model.predict(test, num_iteration = 200)
    return predictions

In [14]:
def dummyBuilding(train, test):
    ix = 0
    for df in [train, test]:
        df_pickup_cluster = pd.get_dummies(df['pickup_cluster'], prefix = 'p', prefix_sep = '_')
        df_dropoff_cluster = pd.get_dummies(df['dropoff_cluster'], prefix = 'd', prefix_sep = '_')
        df_wday = pd.get_dummies(df['wday'], prefix = 'd', prefix_sep ='_')
        df_hour = pd.get_dummies(df['hour'], prefix = 'h', prefix_sep = '_')
        df_condition = pd.get_dummies(df['condition'], prefix = 'c', prefix_sep = '_')
        
        
        df.drop(['passenger_count'], axis = 1, inplace = True)
        df.drop(['wday', 'hour', 'condition'], axis = 1, inplace = True)
        if ix == 0: df.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis =1, inplace = True)
        else: df.drop(['pickup_longitude','pickup_latitude'], axis =1, inplace = True)
        
        df_Master = pd.concat([df, df_pickup_cluster, df_dropoff_cluster, 
                              df_wday, df_hour, df_condition], axis = 1)
        del df, df_pickup_cluster, df_dropoff_cluster, df_wday, df_hour, df_condition
        
        if ix == 0: train = df_Master
        else: test = df_Master
        ix += 1
    
    return train, test    
    
        